## Get data from APIs

In [ ]:
import requests
import pandas as pd
import cbsodata

### 1. /v4/gezocht
Search APB’s. Use consecutive methods to retrieve APB’s of a specialized type. Results are sorted on
descending date (newest first).

No results found.
No results found.
No results found.
No results found.


No results found.
